# SparkException: Can only zip RDDs with same number of elements in each partition

Imaginemos que tenemos el siguiente problema para resolver mediante PySpark:

Partimos de dos ficheros de texto, fichero1.txt y fichero2.txt con el siguiente formato:

linea   1       dato1   dato22  dato43
linea   2       dato2   dato23  dato44
linea   3       dato3   dato24  dato45
linea   4       dato4   dato25  dato46
linea   5       dato5   dato26  dato47
linea   6       dato6   dato27  dato48
linea   7       dato7   dato28  dato49
linea   8       dato8   dato29  dato50
linea   9       dato9   dato30  dato51
......


y fichero2.txt



linea   1       otrodato1       otrodato22      otrodato43
linea   2       otrodato2       otrodato23      otrodato44
linea   3       otrodato3       otrodato24      otrodato45
linea   4       otrodato4       otrodato25      otrodato46
linea   5       otrodato5       otrodato26      otrodato47
linea   6       otrodato6       otrodato27      otrodato48
linea   7       otrodato7       otrodato28      otrodato49
linea   8       otrodato8       otrodato29      otrodato50
linea   9       otrodato9       otrodato30      otrodato51
...


Cada uno de ellos con el mismo número de líneas (110)


El objetivo es unir cada una de las líneas del fichero1 con la línea correspondiente del fichero2. De forma que el resultado fuese:


[ ( linea   1       dato1   dato22  dato43 ), ( linea   1       otrodato1       otrodato22      otrodato43) ]

Para realizar esta operación en pySpark podemos usar la acción zip:


In [ ]:
distData1 = sc.textFile('file://///RUTA_AL_FICHERO1.TXT',40)
distData2 = sc.textFile('file:////RUTA_AL_FICHERO2.TXT',40)
resultado = distData1.zip (distData2)
resultado.collect()

Pero nos puede arrojar el siguiente error:

Can only zip with RDD which has the same number of partitions

El problema parece consistir en que la función Zip: " Assumes that the two RDDs have the same number of partitions and the same number of elements in each partition" y eso es algo que no siempre se produce.

En concreto, para ficheros grandes (1K líneas) poniendo particionado automático suele dar el problema. Si en vez de ficheros usamos listas o ficheros pequeños no parece ocurrir. No parece tener un patrón definido. Lo único que podemos es saber que no lo hace de forma correcta cuando se produce el error anterior.

Para comprobar si el particionado se ha hecho de forma correcta, podemos recurrir al siguiente código:


In [ ]:
print distData1.glom().map(lambda d: len(d)).collect()
print distData2.glom().map(lambda d: len(d)).collect()

Lo que hacemos es recorrer cada una de las particiones de los datos paralelizados y mostrar el número de elementos que tiene cada una. De esta forma podemos comprobar dónde se encuentra el error.

[3, 3, 3, 2, 3, 2, 3, 2, 4, 3, 2, 3, 3, 2, 3, 2, 4, 3, 2, 3, 2, 3, 3, 2, 4, 2, 3, 3, 2, 3, 2, 3, 4, 2, 3, 3, 2, 3, 2, 3, 1]
[3, 3, 3, 2, 3, 2, 3, 3, 3, 3, 3, 2, 3, 2, 3, 2, 4, 3, 3, 2, 3, 2, 3, 2, 4, 3, 3, 2, 3, 2, 3, 2, 4, 3, 3, 2, 3, 2, 3, 3]

En este caso vemos que la octava posición intentará combinar dos elementos con tres, lo que producirá un error.

Si ocurre esto podemos solucionarlo haciendo algo un poco más menos eficiente que el zip: combinarlos con un índice y agruparlos por clave:


In [ ]:
data1_index=distData1.zipWithIndex().map(lambda (a,b): (b,a))
data2_index=distData2.zipWithIndex().map(lambda (a,b): (b,a))
final = data1_index.join(data2_index)
print final.collect()